In [ ]:
%%HTML
<!-- Mejorar visualización en proyector -->
<style>
.rendered_html {font-size: 1.2em; line-height: 150%;}
div.prompt {min-width: 0ex; padding: 0px;}
.container {width:95% !important;}
</style>

In [ ]:
%autosave 0
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from tqdm import tqdm_notebook
import torch
import torchvision

### Dataset y dataloaders

In [ ]:
from torch.utils.data import DataLoader, Subset
import sklearn.model_selection

mnist_train_data = torchvision.datasets.MNIST('dataset', train=True, download=True, 
                                              transform=torchvision.transforms.ToTensor())

fig, ax = plt.subplots(1, 10, figsize=(8, 2), tight_layout=True)
for k in range(10):
    image, label = mnist_train_data[k]
    ax[k].imshow(image.numpy()[0, :, :], cmap=plt.cm.Greys_r)
    ax[k].axis('off');
    ax[k].set_title(label)
    
sss = sklearn.model_selection.StratifiedShuffleSplit(train_size=0.6)
train_idx, valid_idx = next(sss.split(np.zeros(len(mnist_train_data)), mnist_train_data.targets))

train_dataset = Subset(mnist_train_data, train_idx)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
valid_dataset = Subset(mnist_train_data, valid_idx)
valid_loader = DataLoader(valid_dataset, shuffle=False, batch_size=256)

mnist_test_data = torchvision.datasets.MNIST('dataset', train=False, download=True,
                                             transform=torchvision.transforms.ToTensor())
test_loader = DataLoader(mnist_test_data, shuffle=False, batch_size=256)

### Autoencoder

In [ ]:
class mi_autoencoder(torch.nn.Module):
    
    def __init__(self):
        super(mi_autoencoder, self).__init__()
        # Llenar
    
    def encode(self, x):
        # Llenar        
        return x
        
    def decode(self, z):
        # Llenar
        return z
    
    def forward(self, x):
        z = self.encode(x)
        return self.decode(z)

### Entrenamiento

In [ ]:
nnet = mi_autoencoder()
optimizer = torch.optim.Adam(nnet.parameters(), lr=1e-3)
criterion = torch.nn.BCEWithLogitsLoss(reduction='sum')
use_gpu = False
nepochs = 5
if use_gpu:
    nnet = nnet.cuda()

import time
from torch.utils.tensorboard import SummaryWriter
# tensorboard --logdir=/tmp/tensorboard
current_time = str(time.time_ns())
train_writer = SummaryWriter("/tmp/tensorboard/ae/"+current_time+"/train/", flush_secs=10)
valid_writer = SummaryWriter("/tmp/tensorboard/ae/"+current_time+"/valid/", flush_secs=10)

for epoch in range(nepochs): 
    # Train
    epoch_loss = 0.0
    for mbdata, mblabel in train_loader:
        if use_gpu:
            mbdata, mblabel = mbdata.cuda(), mblabel.cuda()
        prediction = nnet(mbdata)
        optimizer.zero_grad()        
        loss = criterion(prediction, mbdata)
        epoch_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_writer.add_scalar('Loss', epoch_loss/len(train_idx), epoch)
    # Validation    
    epoch_loss = 0.0
    for mbdata, mblabel in valid_loader:
        if use_gpu:
            mbdata, mblabel = mbdata.cuda(), mblabel.cuda()
        prediction = nnet(mbdata)
        loss = criterion(prediction, mbdata)
        epoch_loss += loss.item()
    valid_writer.add_scalar('Loss', epoch_loss/len(valid_idx), epoch)

if use_gpu:
    nnet = nnet.cpu()
    
train_writer.close()
valid_writer.close()

### Visualización de las reconstrucciones

In [ ]:
fig, axs = plt.subplots(2, 10, figsize=(8, 3))
P = np.random.permutation(10000)

for i in range(10):
    image, label = mnist_test_data[P[i]]
    hat_image = torch.nn.Sigmoid()(nnet.forward(image.unsqueeze(0)))
    axs[0, i].matshow(image.numpy()[0, :, :], cmap=plt.cm.Greys_r)
    axs[0, i].axis('off');
    axs[0, i].set_title(label)
    axs[1, i].matshow(hat_image.detach().numpy()[0, 0, :, :], cmap=plt.cm.Greys_r)
    axs[1, i].axis('off');
plt.tight_layout();

### Visualización del espacio latente

In [ ]:
fig = plt.figure(figsize=(10, 4), dpi=80)
ax_main = plt.subplot2grid((2, 3), (0, 0), colspan=2, rowspan=2)
ax_ori = plt.subplot2grid((2, 3), (0, 2))
ax_rec = plt.subplot2grid((2, 3), (1, 2))
ax_ori.axis('off'); ax_rec.axis('off');

z_eval = np.zeros(shape=(len(mnist_test_data), 2))
for i, (image, label) in enumerate(test_loader):
    z_eval[i*256:(i+1)*256] = nnet.encode(image).detach().numpy()

for t in range(10):
    ax_main.scatter(z_eval[mnist_test_data.targets.numpy() == t, 0], 
                    z_eval[mnist_test_data.targets.numpy() == t, 1], s=5, alpha=0.5, label=str(t))
ax_main.legend();

def onclick(event):
    z_closest = [event.xdata, event.ydata]
    idx = np.argmin(np.sum((z_eval - z_closest)**2, axis=1))
    image, label = mnist_test_data[idx]
    hat_image = torch.nn.Sigmoid()(nnet.forward(image.unsqueeze(0)))
    ax_ori.matshow(mnist_test_data[idx][0].numpy()[0, :, :], cmap=plt.cm.Greys_r)
    ax_rec.matshow(hat_image.detach().numpy()[0, 0, :, :], cmap=plt.cm.Greys_r)
    
cid = fig.canvas.mpl_connect('button_press_event', onclick)

### Backup: Versión antigua en tensorflow

In [ ]:
"""

Use this to select GPU and limit the memory usage

"""
environ["CUDA_VISIBLE_DEVICES"] = "0"
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3

nepochs = 20
batch_size = 32
metrics = np.zeros(shape=(nepochs, 2))

tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 784])
#with tf.variable_scope('encoder'):
#    h = tf.layers.dense(inputs=x, units=64, use_bias=True, activation=tf.nn.relu)
with tf.variable_scope('latent'):
    z = tf.layers.dense(inputs=x, units=2, use_bias=True, activation=None)
with tf.variable_scope('decoder'):
    #h = tf.layers.dense(inputs=z, units=64, use_bias=True, activation=tf.nn.relu)
    y = tf.layers.dense(inputs=z, units=784, use_bias=True, activation=None)
    x_hat = tf.nn.sigmoid(y)

cross_entropy = tf.nn.sigmoid_cross_entropy_with_logits(logits=y, labels=x)
reg = tf.reduce_mean(tf.norm(z, ord=2))
loss = tf.reduce_mean(cross_entropy)
beta = 0.0
train_step = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(loss + beta*reg)

fig, ax = plt.subplots(1, 1, figsize=(5, 3), dpi=80)
ax.plot(0, linewidth=2, alpha=0.75, label='Train'); 
ax.plot(0, linewidth=2, alpha=0.75, label='Test');
ax.set_ylabel('Loss'); ax.set_xlabel('Epoch')
line1, line2 = ax.lines
plt.legend(); plt.grid(); plt.tight_layout();

with tf.Session(config=config) as sess:    
    tf.global_variables_initializer().run()
    for epoch in tqdm_notebook(range(nepochs), desc='Train epoch'):
        for _ in range(int(mnist.train.images.shape[0]/batch_size)):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step, feed_dict={x: batch_xs})
        metrics[epoch, 0] = sess.run(loss, feed_dict={x: mnist.train.images})
        metrics[epoch, 1] = sess.run(loss, feed_dict={x: mnist.test.images})
        if epoch > 0:
            line1.set_data(range(epoch+1), metrics[:epoch+1, 0])
            line2.set_data(range(epoch+1), metrics[:epoch+1, 1])
            ax.set_xlim([0, epoch])
            ax.set_ylim([np.amin(metrics[:epoch+1,:]), np.amax(metrics[:epoch+1,:])])
            fig.canvas.draw()
    z_eval = sess.run(z, feed_dict={x: mnist.test.images})
    x_hat_eval = sess.run(x_hat, feed_dict={x: mnist.test.images})
